# Improvement of templates

## Static assertions

The new `static_assert` directive allows you to check a condition **during compilation** and to display the message of your choice in case of failure. It is typically used to test a type used in a template:

In [1]:
template <typename Int>
struct Rational {
    static_assert(sizeof(Int)>=4,"Underlying type size is not long enough\n") ;
    Int numerator ;
    Int denominator ;
} ; 

In [2]:
Rational<int> r1 ;
Rational<short> r2 ;

input_line_7:3:5: error: static_assert failed "Underlying type size is not long enough\n"
    static_assert(sizeof(Int)>=4,"Underlying type size is not long enough\n") ;
    ^             ~~~~~~~~~~~~~~
input_line_8:3:17: note: in instantiation of template class '__cling_N52::Rational<short>' requested here
Rational<short> r2 ;
                ^


Interpreter Error: 

In [5]:
template <typename T, int size>
struct Array {
    static_assert(size>0,"Array size must be strictly positive\n") ;
    T data[size] ;
} ; 

In [6]:
int const n { 2 } ;
// ...
Array<int,n> a1 ;
Array<int,n-1> a2 ;
Array<int,n-2> a3 ;

input_line_11:3:5: error: static_assert failed "Array size must be strictly positive\n"
    static_assert(size>0,"Array size must be strictly positive\n") ;
    ^             ~~~~~~
input_line_12:6:16: note: in instantiation of template class '__cling_N56::Array<int, 0>' requested here
Array<int,n-2> a3 ;
               ^


Interpreter Error: 

## Alias template

It becomes possible to define some kinds of `typedef` with template parameters.

The new keyword for this is `using`.

For example, it allows to give a name to a partial specialization of a class template:

In [ ]:
template <typename T, typename U>
class Pair { public : T x ; U y ; } ;

template <typename U>
using PairInt = Pair<int,U> ;

PairInt<double> pid ; // equivalent to Pair<int,double>

In [ ]:
template <typename T>
using MapInt = std::map<int,T> ;

MapInt<double> md ; // equivalent to std::map<int,double>

The `using` statement should now replace` typedef` under all circumstances. It is considered more readable. Below are some examples of equivalent instructions.

In [ ]:
#include <list>
#include <map>
#include <iostream>

In [ ]:
using real = float ; // typedef float real ;
    
using my_map = std::map<std::vector<int>,std::list<float>> ;
using my_citr = my_map::const_iterator ;
    
using fptr = void(*)(int) ;

Starting from C ++ 14, the standard library includes some **`_t`** shortcuts for the templates that are used to manipulate on types. For example:

In [18]:
template<typename T>
using remove_pointer_t = typename std::remove_pointer<T>::type ;

Note however that with the appearance of `auto`, it is less and less necessary to use a type alias to lighten the writing of user code.

## Variable templates

C++14 introduces the possibility of making template of variables. The example usually given is a constant variable pi, that we could define with different precision for all the predefined types.

In [4]:
%%file tmp.templates.cpp

#include <iostream>

template<typename T>
const T pi = T(3.1415926535897932385) ;

template<typename T>
T circular_area(T a_r)
 { return pi<T> * a_r * a_r ; }

int main()
 {
  std::cout.precision(18) ;
  std::cout << "double : " << circular_area(1.) << std::endl ;
  std::cout << "float  : " << circular_area(1.f) << std::endl ;
  std::cout << "int    : " << circular_area(1) << std::endl ;
 }

Writing tmp.templates.cpp


In [5]:
!rm -f tmp.templates.exe && g++ -std=c++17 tmp.templates.cpp -o tmp.templates.exe

In [6]:
!./tmp.templates.exe

double : 3.14159265358979312
float  : 3.14159274101257324
int    : 3


Starting from C++17, the standard library includes some **`_v` shortcuts** for the templates that are testing type properties. For example:

In [7]:
%%file tmp.templates.cpp

#include <type_traits>

template< class T >
bool const is_integral_v = std::is_integral<T>::value ;

template <typename IntegralT>
struct Rational {
    static_assert(is_integral_v<IntegralT>,"Bad IntegralT") ;
    IntegralT numerator ;
    IntegralT denominator ;
} ;

int main() {
  Rational<int> r1 ;
  Rational<double> r2 ;
}

Writing tmp.templates.cpp


In [28]:
!rm -f tmp.templates.exe && g++ -std=c++17 tmp.templates.cpp -o tmp.templates.exe

tmp.templates.cpp: In instantiation of 'struct Rational<double>':
tmp.templates.cpp:16:20:   required from here
tmp.templates.cpp:9:19: error: static assertion failed: Bad IntegralT
    9 |     static_assert(is_integral_v<IntegralT>,"Bad IntegralT") ;
      |                   ^~~~~~~~~~~~~~~~~~~~~~~~


## Help with instantiating templates

Heavy use of templates can lead to the duplication of these templates in all binaries. C ++ 11 allows the developer to help the compiler by indicating which file to place a template in, and which not to. It servers as a tool to fight against "code bloat".

#### Forbid the instantiation of a template in the current file

``` cpp
extern template class Point<int> ;
```

#### Force instantiation of a template in the current file

``` cpp
template class Point<int> ; // Point<int> will be instantiated in the current file
```

## Class template argument deduction (C++17)

For a template function, the compiler was already able to infer the type parameters from the arguments. Starting with C++17, it is also ok working with classes.

## Variadic templates

C ++ 11 introduces the possibility of defining templates with a variable number of parameters. Such a template is generally defined using a general statement, a **recursive specialisation** and a terminal specialisation.

#### Example of tuple by composition

In [29]:
%%file tmp.tuples.h

template <typename... Types> struct Tuple ;

template <typename T, typename... Types>
struct Tuple<T, Types...>
 {
  T data ;
  Tuple<Types...> others ;
 } ;

template <> struct Tuple<> {} ;

Writing tmp.tuples.h


In [30]:
%%file tmp.templates.cpp

#include <iostream>
#include <string>
#include "tmp.tuples.h"

int main()
 {
  Tuple<int,double,std::string> tuple { 42, 3.14, "bonjour" } ;
  std::cout << tuple.data << std::endl ;
  std::cout << tuple.others.data << std::endl ;
  std::cout << tuple.others.others.data << std::endl ;
 }

Overwriting tmp.templates.cpp


In [31]:
!rm -f tmp.templates.exe && g++ -std=c++17 tmp.templates.cpp -o tmp.templates.exe

In [32]:
!./tmp.templates.exe

42
3.14
bonjour


#### The ellipsis operator `...` also allows to define a pack of arguments in a function

In [33]:
#include <iostream>
void print( int i, double d )
 { std::cout<<i<<" "<<d<<std::endl ; }  

In [36]:
template <typename Function, typename... Args>
void apply( Function f, Args... args )
 {
  std::cout<<"applying"<<std::endl ;
  f(args...) ;
 }  

In [37]:
apply(print,42,3.14) ;

applying
42 3.14


## Questions ?

# Exercise

Complete the `make_ptr` function, imitation of `std::make_shared`.

In [ ]:
%%file tmp.templates.cpp

#include <memory>
#include <iostream>

class MyData
 {
  public :
    MyData( int i, double d ) : m_i {i}, m_d {d}
     { std::cout<<"MyData::MyData()"<<std::endl ; }
    int i() { return m_i ; }
    double d() { return m_d ; }
    ~MyData()
     { std::cout<<"MyData::~MyData()"<<std::endl ; }
  private :
    int m_i ;
    double m_d ;
 } ;

void display( std::shared_ptr<MyData> data_ptr )
 { std::cout<<data_ptr->i()<<" "<<data_ptr->d()<<std::endl ; }  

template <typename T, typename... Args>
std::shared_ptr<T> make_ptr( Args... args )
 { return ??? ; }  

int main()
 {
  auto data_ptr {make_ptr<MyData>(42,3.14)} ;
  print(data_ptr) ;
  return 0 ;
 }

In [ ]:
!rm -f tmp.templates.exe && g++ -std=c++17 tmp.templates.cpp -o tmp.templates.exe

In [ ]:
!./tmp.templates.exe

© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*